In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score

In [2]:
data = pd.read_csv('train.tsv',sep='\t')
tags = ["part-time-job", "full-time-job", "hourly-wage", "salary", "associate-needed", "bs-degree-needed", "ms-or-phd-needed", "licence-needed", "1-year-experience-needed", "2-4-years-experience-needed", "5-plus-years-experience-needed", "supervising-job"]
data.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [5]:
y = []

for tag in tags:
    y_curr = []
    for tags_list in data.tags:
        if (type(tags_list) != float) and (tag in tags_list):
            y_curr.append(1)
        else:
            y_curr.append(0)
    y.append(y_curr)
y = np.array(y).T

The same score function as used at HackerRank contest

In [6]:
def score(predictions, true_values):
    if predictions.shape != true_values.shape:
        print 'Error: dimensions must be the same'
        return
    STP, SFP, STN, SFN = 0, 0, 0, 0
    for i in xrange(len(tags)):
        STP += sum([1 if true_values[j,i]==1 and predictions[j,i]==1 else 0 for j in xrange(len(predictions))])
        SFP += sum([1 if true_values[j,i]==0 and predictions[j,i]==1 else 0 for j in xrange(len(predictions))])
        STN += sum([1 if true_values[j,i]==0 and predictions[j,i]==0 else 0 for j in xrange(len(predictions))])
        SFN += sum([1 if true_values[j,i]==1 and predictions[j,i]==0 else 0 for j in xrange(len(predictions))])
    P = float(STP)/(STP+SFP)
    R = float(STP)/(STP+SFN)
    return P, R, 2*P*R/(P+R)

In [ ]:
score()